In [1]:
!pip install giskard openai ragas sentence_transformers --upgrade scikit-learn hdbscan  giskard[llm]

  Using cached ragas-0.2.5-py3-none-any.whl.metadata (8.0 kB)
^C
ERROR: Operation cancelled by user


In [ ]:
import giskard
from typing import Sequence, Optional
from giskard.llm.client import set_default_client
from giskard.llm.client.base import LLMClient, ChatMessage
from openai import OpenAI


# Create a custom client by extending the LLMClient class
class MyLLMClient():
        
    def complete(
            self,
            messages: Sequence[ChatMessage],
            temperature: float = 1,
            max_tokens: Optional[int] = None,
            caller_id: Optional[str] = None,
            seed: Optional[int] = None,
            format=None,
    ) -> ChatMessage:
      
        XAI_API_KEY = "PUNYAMU"
        client = OpenAI(
            api_key=XAI_API_KEY,
            base_url="https://api.x.ai/v1",
        )

        prompt_message = [
                {"role": "system", "content": "You are Grok, a medical chatbot"},
            ]

        for msg in messages:
            if msg.role.lower() == "assistant":
                prompt_message.append({
                    "role": "assistant", "content": msg.content
                })
            else:
                 prompt_message.append({
                    "role": "user", "content": msg.content
                })

        
        completion = client.chat.completions.create(
            model="grok-beta",
            messages=prompt_message,
            temperature= 0.25
        )

        ans = completion.choices[0].message.content

        return ChatMessage(role="assistant", content=ans)


llm_client = MyLLMClient()

# Set the default client
set_default_client(llm_client)

In [89]:
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs,
    query_instruction="Represent this sentence for searching relevant passages:"
)

embeddings.query_instruction = "Represent this sentence for searching relevant passages:"


In [90]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader


medrag_textbook_loader = HuggingFaceDatasetLoader("MedRAG/textbooks", "contents")

In [91]:
medrag_textbook_loader_data = medrag_textbook_loader.load()

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [92]:
type(medrag_textbook_loader_data)

list

In [60]:
!pip install --upgrade scipy

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [95]:
medrag_textbook_data =[] 
for row in medrag_textbook_loader_data:
    medrag_textbook_data.append(row.page_content)

In [96]:
from giskard.llm.embeddings import BaseEmbedding 
class EmbeddingsWrapper(BaseEmbedding):
    def __init__(self, embeddings):
        self.embeddings = embeddings

    def embed(self, texts):
        return self.embeddings.embed_documents(texts)

wrapped_embeddings = EmbeddingsWrapper(embeddings)

In [97]:
import random

small_medrag_textbook_data = random.sample(medrag_textbook_data, 5000)
print(len(small_medrag_textbook_data))

5000


In [98]:
import pandas as pd
medrag_textbook_loader_df = pd.DataFrame(small_medrag_textbook_data, columns=["page_content"])

In [99]:
medrag_textbook_loader_df.head()

,page_content
0,"""Cell_Biology_Alberts. Figure 13\u201346 The r..."
1,"""Physiology_Levy. 12. How does the organizatio..."
2,"""Immunology_Janeway. and cytoplasmic segments ..."
3,"""Pediatrics_Nelson. Phimosis is rarely symptom..."
4,"""Immunology_Janeway. was blocked. Normal numbe..."


In [66]:
from giskard.rag import generate_testset, KnowledgeBase

knowledge_base = KnowledgeBase.from_pandas(medrag_textbook_loader_df, columns=["id","page_content"], embedding_model=wrapped_embeddings)


In [67]:
testset = generate_testset(
    knowledge_base,
    num_questions=550,
    language='en',  # optional, we'll auto detect if not provided
    agent_description="A medical chatbot", # helps generating better questions
)

Generating questions:   0%|          | 0/550 [00:00<?, ?it/s]

In [68]:
# Save the generated testset
testset.save("my_testset.jsonl")

# You can easily load it back
from giskard.rag import QATestset

loaded_testset = QATestset.load("my_testset.jsonl")

In [69]:
test_df = loaded_testset.to_pandas()


In [70]:
test_df.head()

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
bafdc266-3a82-4ddf-9484-08c7a40d747b,What is the smallest detail that can be resolv...,The light microscope can resolve details 0.2 μ...,"Document 434: page_content: ""Cell_Biology_Albe...",[],"{'question_type': 'simple', 'seed_document_id'..."
d53903b3-7a45-436d-82e1-956404718cc1,What should be considered for patients with ch...,Patients who do not respond to conventional th...,"Document 4390: page_content: ""InternalMed_Harr...",[],"{'question_type': 'simple', 'seed_document_id'..."
c0c48ffe-998c-4237-952c-c73b6912746c,Can heparin treatment for deep-vein thrombosis...,"Yes, full-dose heparin can be restarted severa...","Document 2207: page_content: ""Obstentrics_Will...",[],"{'question_type': 'simple', 'seed_document_id'..."
61c2056d-fb20-4f56-9002-3c2c01f76fef,What is the function of BTLA when it and HVEM ...,When BTLA and HVEM are co-expressed on the sam...,"Document 517: page_content: ""Immunology_Janewa...",[],"{'question_type': 'simple', 'seed_document_id'..."
83e28e86-fc81-4180-bfd4-c424816d60f5,What is the principal clinical usefulness of H...,The principal clinical usefulness of HBeAg in ...,"Document 2933: page_content: ""InternalMed_Harr...",[],"{'question_type': 'simple', 'seed_document_id'..."


In [100]:
excluded_docs = set(small_medrag_textbook_data)


In [101]:
import random
remaining_docs = [doc for doc in medrag_textbook_data if doc not in excluded_docs]

small_medrag_textbook_data2 = random.sample(remaining_docs, 5000)
print(len(small_medrag_textbook_data2))

5000


In [102]:
import pandas as pd
medrag_textbook_loader_df_2 = pd.DataFrame(small_medrag_textbook_data2, columns=["page_content"])

In [103]:
from giskard.rag import generate_testset, KnowledgeBase

knowledge_base_2 = KnowledgeBase.from_pandas(medrag_textbook_loader_df_2, columns=["id","page_content"], embedding_model=wrapped_embeddings)


In [104]:
testset = generate_testset(
    knowledge_base_2,
    num_questions=550,
    language='en',  # optional, we'll auto detect if not provided
    agent_description="A medical chatbot", # helps generating better questions
)

Generating questions:   0%|          | 0/550 [00:00<?, ?it/s]

In [105]:
# Save the generated testset
testset.save("my_testset2.jsonl")

# You can easily load it back
from giskard.rag import QATestset

loaded_testset = QATestset.load("my_testset2.jsonl")

In [71]:
test_df.iloc[2]

question                Can heparin treatment for deep-vein thrombosis...
reference_answer        Yes, full-dose heparin can be restarted severa...
reference_context       Document 2207: page_content: "Obstentrics_Will...
conversation_history                                                   []
metadata                {'question_type': 'simple', 'seed_document_id'...
Name: c0c48ffe-998c-4237-952c-c73b6912746c, dtype: object

In [73]:
test_df.shape[0]

550